In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import sklearn as sk

%matplotlib inline

In [2]:
diag = pd.read_csv('/Users/shufei/Novartis/APLD Raw Data/DIAG.txt', 
                   dtype={"PATIENT_ID": int,"CLAIM_ID": object,"CLAIM_TYP_CD": object,"SERVICE_DATE": str,"MONTH_ID": int,
                          "DIAGNOSIS_CODE": object,"DIAG_VERS_TYP_ID": int,"DIAG_CD_POSN_NBR": int,"PROVIDER_ID": object,
                          "RESTATE_FLAG": object,"FLEXIBLE_FLD_1_CHAR": object,"FLEXIBLE_FLD_2_CHAR": object}, 
                   sep="|")
#sort by date
diag.sort_values(by=['PATIENT_ID','MONTH_ID','SERVICE_DATE'],ascending=(False,True,True),inplace=True)
diag.reset_index(drop=True,inplace=True)
diag.head()

,PATIENT_ID,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE,MONTH_ID,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,PROVIDER_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR
0,1734677597,117121790014,P,12/03/2014,201412,174.9,1,1,7853274,0,1558366674,NaN
1,1734677597,117056938833,P,12/03/2014,201412,174.9,1,1,7032941,0,1730184417,NaN
2,1734677597,117046044155,P,12/03/2014,201412,174.9,1,1,18396046,0,1649264300,NaN
3,1734677597,10655089494000222552,I,12/03/2014,201412,174.4,1,1,6954249,0,1225091226,NaN
4,1734677597,10655089456700328900,I,12/03/2014,201412,174.4,1,1,6954249,0,1225091226,NaN


In [3]:
# read bc_sn_Code
bc_sn_Code =pd.read_csv('/Users/shufei/Novartis/BC_SN ICD Code.csv',header=0)
bc_sn_Code.rename(index=str, columns={"diagnosis_cd": "DIAGNOSIS_CODE",'indication_cd':'INDICATION_CODE'},inplace=True)
bc_sn_Code.head()

,DIAGNOSIS_CODE,INDICATION_CODE,version,sub_indication_cd
0,174,BC,ICD_9,NaN
1,174.0,BC,ICD_9,NaN
2,174.1,BC,ICD_9,NaN
3,174.2,BC,ICD_9,NaN
4,174.3,BC,ICD_9,NaN


In [4]:
#merge two dataframe get bc or sn 
DIAG_DATA = pd.merge(diag,bc_sn_Code, how="left", on = "DIAGNOSIS_CODE")
DIAG_DATA.head()

,PATIENT_ID,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE,MONTH_ID,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,PROVIDER_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR,INDICATION_CODE,version,sub_indication_cd
0,1734677597,117121790014,P,12/03/2014,201412,174.9,1,1,7853274,0,1558366674,NaN,BC,ICD_9,NaN
1,1734677597,117056938833,P,12/03/2014,201412,174.9,1,1,7032941,0,1730184417,NaN,BC,ICD_9,NaN
2,1734677597,117046044155,P,12/03/2014,201412,174.9,1,1,18396046,0,1649264300,NaN,BC,ICD_9,NaN
3,1734677597,10655089494000222552,I,12/03/2014,201412,174.4,1,1,6954249,0,1225091226,NaN,BC,ICD_9,NaN
4,1734677597,10655089456700328900,I,12/03/2014,201412,174.4,1,1,6954249,0,1225091226,NaN,BC,ICD_9,NaN


In [5]:
# drop null value: neither BC nor SN
df = DIAG_DATA.drop(DIAG_DATA[pd.isnull(DIAG_DATA['INDICATION_CODE'])].index, inplace=False)
# only consider BC
df = df.loc[df['INDICATION_CODE']=='BC',] 
# get unique patient, also earliest
df3 = df.drop_duplicates(subset = ["PATIENT_ID"])
#rename SERVICE_DATE to BC_DIAG_DATE
df3.rename(index=str, columns={"SERVICE_DATE":"BC_DIAG_DATE"},inplace=True)

DIAG_BC=df3[["PATIENT_ID","CLAIM_ID","BC_DIAG_DATE","INDICATION_CODE"]]

/Users/shufei/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [6]:
# get sub_indication_cd isn't null, equal to all sn patient
df2 = DIAG_DATA.drop(DIAG_DATA[pd.isnull(DIAG_DATA['sub_indication_cd'])].index,inplace=False)
# get unique patient, also earliest
df4 = df2.drop_duplicates(subset=["PATIENT_ID"])
# rename SERVICE_DATE to SN_DIAG_DATE
df4.rename(index=str, columns={"SERVICE_DATE":"SN_DIAG_DATE"},inplace=True)


DIAG_SN=df4[["PATIENT_ID","CLAIM_ID","SN_DIAG_DATE","sub_indication_cd"]]

In [7]:
# patient both diagnose bc and sn
DIAG_BC_SN = pd.merge(DIAG_BC,DIAG_SN, how="inner", on = "PATIENT_ID")
DIAG_BC_SN.astype({"BC_DIAG_DATE":"str","SN_DIAG_DATE":"str"}).dtypes
DIAG_BC_SN

,PATIENT_ID,CLAIM_ID_x,BC_DIAG_DATE,INDICATION_CODE,CLAIM_ID_y,SN_DIAG_DATE,sub_indication_cd
0,1734571640,10809134400909755016,07/07/2016,BC,10809132639600052060,06/29/2017,META_STATIC
1,1734568746,10420104060601768535,10/18/2017,BC,10420104060601768535,10/18/2017,META_STATIC
2,1734477535,10419085891401043928,11/16/2016,BC,10490091223101872407,02/22/2017,META_STATIC
3,1734453693,10490087468108851479,11/10/2016,BC,10193097791800464057,07/06/2017,META_STATIC
4,1734310400,10809132746414858841,03/31/2017,BC,10655093829800225278,04/17/2017,META_STATIC
...,...,...,...,...,...,...,...
8820,29632133,117637965189,01/08/2015,BC,10241115628500768300,03/15/2018,META_STATIC
8821,29613658,10420079552600119824,07/20/2016,BC,10420180681508131215,11/08/2019,META_STATIC
8822,29581050,10655088723600289420,12/08/2016,BC,10655088723600289420,12/08/2016,META_STATIC
8823,29544598,112824059986,05/19/2014,BC,112436878075,05/29/2014,META_STATIC


In [8]:
# function days to get the date differences
def days(str1,str2):
    date1=datetime.datetime.strptime(str1,"%m/%d/%Y")
    date2=datetime.datetime.strptime(str2,"%m/%d/%Y")
    num=(date1-date2).days
    return num

# date diff between SN_DIAD_DATE and BC_DIAG_DATE
def date_diff(row):
    day1=row["SN_DIAG_DATE"]
    day2=row["BC_DIAG_DATE"]
    day = days(day1,day2) 
    return day

# new column DATE_DIFF
DIAG_BC_SN["DATE_DIFF"]=DIAG_BC_SN.apply(lambda x:date_diff(x),axis=1)

In [9]:
# define mBC
def mbc_confirm(row):
    if row["DATE_DIFF"] >= -30:
        part1="mBC"
    else:
        part1=""
    return part1
 
# new column mBC
DIAG_BC_SN["mBC_indication_code"]=DIAG_BC_SN.apply(lambda x:mbc_confirm(x),axis=1)



In [10]:
DIAG_BC_SN.head()

,PATIENT_ID,CLAIM_ID_x,BC_DIAG_DATE,INDICATION_CODE,CLAIM_ID_y,SN_DIAG_DATE,sub_indication_cd,DATE_DIFF,mBC_indication_code
0,1734571640,10809134400909755016,07/07/2016,BC,10809132639600052060,06/29/2017,META_STATIC,357,mBC
1,1734568746,10420104060601768535,10/18/2017,BC,10420104060601768535,10/18/2017,META_STATIC,0,mBC
2,1734477535,10419085891401043928,11/16/2016,BC,10490091223101872407,02/22/2017,META_STATIC,98,mBC
3,1734453693,10490087468108851479,11/10/2016,BC,10193097791800464057,07/06/2017,META_STATIC,238,mBC
4,1734310400,10809132746414858841,03/31/2017,BC,10655093829800225278,04/17/2017,META_STATIC,17,mBC


In [11]:
DIAG_BC_SN.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8825 entries, 0 to 8824
Data columns (total 9 columns):
PATIENT_ID             8825 non-null int64
CLAIM_ID_x             8825 non-null object
BC_DIAG_DATE           8825 non-null object
INDICATION_CODE        8825 non-null object
CLAIM_ID_y             8825 non-null object
SN_DIAG_DATE           8825 non-null object
sub_indication_cd      8825 non-null object
DATE_DIFF              8825 non-null int64
mBC_indication_code    8825 non-null object
dtypes: int64(2), object(7)
memory usage: 689.5+ KB
